In [2]:
import requests
import sys
import pandas as pd
import random
import numpy as np
import datarobot as dr
import urllib3
from datarobot import Deployment
import time
urllib3.disable_warnings()

API_URL = 'https://cfds-ccm-prod.orm.datarobot.com/predApi/v1.0/deployments/{deployment_id}/predictions'    # noqa
API_KEY = 'NWUyNmYyNmMwOWEyMzcwNTFiMWM5MjI4OlA3dmV0Y3kyclRSTTBWdG01WmpJSnJKVDUwbS8vejRXN3Fjb0oxTitQdTg9'
DATAROBOT_KEY = '544ec55f-61bf-f6ee-0caf-15c7f919a45d'

DEPLOYMENT_ID = '5f76080011bc9b05c5913638'

MAX_PREDICTION_FILE_SIZE_BYTES = 52428800  # 50 MB

In [3]:
endpoint = 'https://app.datarobot.com/api/v2'
# Put your API token here
api_token = API_KEY
dr.Client(token=api_token, endpoint=endpoint, ssl_verify=False)

In [14]:
df = pd.read_csv('../Churn_Telecom/Data/DR_Demo_Telecomms_Churn_Prediction.csv')

### Simulation of new data

In [30]:
df_new = pd.DataFrame()
for i in range(50):
    numeric_features = list(df.select_dtypes(include=np.number).columns.values)
    other_features = list(set(df.columns) - set(numeric_features))

    df_sample = df.sample(frac=0.5, replace=True, random_state=1234)
    df_sample_simlated = df_sample[numeric_features].apply(lambda x: x*random.uniform(0.9, 1.1), axis=1)
    df_sample_simlated[other_features] = df_sample[other_features]

    df_new = df_new.append(df_sample_simlated, ignore_index=True)

df_new = df_new.reset_index()
df_new['association_id']= df_new.index

### Create prediction data

In [28]:
rounds = round(df_new.shape[0]/100)
for i in range(rounds):
    df_pred = df_new[i*100:(i+1)*100-1]
    df_pred.loc[:, 'Total.day.minutes'] = df_pred.loc[:, 'Total.day.minutes']*random.randint(1, 3)
    df_pred.loc[:,'Customer.service.calls'] = df_pred.loc[:,'Customer.service.calls']*random.randint(1, 2)
    json_records = df_pred.to_json(orient ='records') 
    
    
    
    # Set HTTP headers. The charset should match the contents of the file.
    headers = {
        'Content-Type': 'application/json; charset=UTF-8',
        'Authorization': 'Bearer {}'.format(API_KEY),
        'DataRobot-Key': DATAROBOT_KEY,
    }

    url = API_URL.format(deployment_id=DEPLOYMENT_ID)
    # Make API request for predictions
    predictions_response = requests.post(
        url,
        data=json_records,
        headers=headers,
    )
    
    
    actual_records = df_pred[['Churn','association_id']]
    actual_records.columns = ['actual_value','association_id']
    actual_records.loc[:,'association_id'] = actual_records.loc[:,'association_id'].astype(str)

    deployment = Deployment.get(deployment_id=DEPLOYMENT_ID)
    deployment.submit_actuals(actual_records)
    
    print(i)
    time.sleep(3600)

0
1
2
3
4


KeyboardInterrupt: 

In [184]:
actual_records = df_pred[['Churn','association_id']]
actual_records.columns = ['actual_value','association_id']
actual_records.loc[:,'association_id'] = actual_records.loc[:,'association_id'].astype(str)

deployment = Deployment.get(deployment_id=DEPLOYMENT_ID)
deployment.submit_actuals(actual_records)

In [ ]:
json_actual_records = actual_records.to_json(orient ='records')

In [233]:
actual_records

,actual_value,association_id
66700,False,66700
66701,False,66701
66702,False,66702
66703,False,66703
66704,False,66704
...,...,...
66794,False,66794
66795,False,66795
66796,False,66796
66797,True,66797


In [220]:
# Set HTTP headers. The charset should match the contents of the file.
API_URL = 'https://app.datarobot.com/api/v2/deployments/{deployment_id}/actuals/fromJSON/'    # noqa
headers = {
    'Content-Type': 'application/json; charset=UTF-8',
    'Authorization': 'Bearer {}'.format(API_KEY),
    'DataRobot-Key': DATAROBOT_KEY,
}
url = API_URL.format(deployment_id=DEPLOYMENT_ID)
# Make API request for predictions
predictions_response = requests.post(
    url, 
    json=json_actual_records, 
    headers=headers,
)